In [1]:
from collections import Counter

**Выполнил**: Копин Борис

Простой спелчекер:
1. Строим словарь из текстов на lenta.ru https://cloud.mail.ru/public/FnMq/qCNif6bFG/all- datasets/samples/sample3/
2. Реализуем функцию подсчета расстояния Левенштейна
3. Делаем спелчекер с помощью поиска ближайших по Левенштейну слов
4. Делаем спелчекер путем генерации ближайших слов

In [2]:
X_test = [u"оцинил", u"роботу", u"новвых", u"самалетав", u"и", u"виртолтов", u"в", u"сирийи",]

In [3]:
PATH_TO_DATASET = "./lenta_words.txt"

## 1. Строим словарь из текстов на lenta.ru

In [4]:
def load_dictionary(path_to_gold):
    words = None
    with open(path_to_gold, 'r') as f:
        words = f.read().splitlines()
        
    return Counter(words)

In [5]:
DICTIONARY = load_dictionary(PATH_TO_DATASET)

In [6]:
len(list(DICTIONARY.keys()))

463163

In [7]:
def most_frequent(words):
    i_max = 0
    max = -1
    for i, s in enumerate(words):
        if max < DICTIONARY.get(s):
            i_max = i
            max = DICTIONARY.get(s)
    return words[i_max]

## 2. Реализуем функцию подсчета расстояния Левенштейна

In [8]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 # j+1 instead of j since previous_row and current_row are one character longer
            deletions = current_row[j] + 1       # than s2
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

## 3. Cпелчекер по Левенштейну слов

In [9]:
def levenshtein_spell_checker(test_word):
    print("checking: " + test_word)

    leve = lambda word: levenshtein(word, test_word)
    
    results = list(map(leve, list(DICTIONARY.keys())))
    minimum = min(results)
    
    siblings = []

    for i, word in enumerate(DICTIONARY):
        if results[i] == minimum or (results[i] == 1 if minimum else False):
            siblings.append(word)
            print("    debug| word: %s  counts: %s  dist: %s" % (word, DICTIONARY.get(word), results[i]))
            
    best_match = most_frequent(siblings) 
    print("  result: " + best_match)
    return best_match

#### Тестируем

In [10]:
levenshtein_spell_checker(X_test[4]);

checking: и
    debug| word: и  counts: 402766  dist: 0
  result: и


In [11]:
levenshtein_spell_checker(X_test[0]);

checking: оцинил
    debug| word: оценил  counts: 437  dist: 1
  result: оценил


In [12]:
levenshtein_spell_checker(X_test[3]);

checking: самалетав
    debug| word: сабалета  counts: 6  dist: 2
    debug| word: самолетам  counts: 81  dist: 2
    debug| word: самолета  counts: 2995  dist: 2
    debug| word: самолетов  counts: 2654  dist: 2
    debug| word: самолетах  counts: 244  dist: 2
  result: самолета


## 4. Cпелчекер путем генерации ближайших слов

In [13]:
LOWER_CASE_LETTERS = {u'а', u'б', u'в', u'г', u'д', u'е', u'ё', u'ж',
                      u'з', u'и', u'й', u'к', u'л', u'м', u'н', u'о',
                      u'п', u'р', u'с', u'т', u'у', u'ф', u'х', u'ц',
                      u'ч', u'ш', u'щ', u'ъ', u'ы', u'ь', u'э', u'ю', u'я',}

In [14]:
len(LOWER_CASE_LETTERS)

33

In [15]:
def generate_by_substitutions(word):
    new_words = []
    
    for i in range(len(word)):
        for l in LOWER_CASE_LETTERS - set([word[i]]):
            new_word = word[:i] + l + word[i+1:]
            new_words.append(new_word)

#     print(len(new_words))
    
    return new_words


def generate_by_deletions(word):
    new_words = []
    
    for i in range(len(word)):
        new_word = word[:i] + word[i+1:]
        new_words.append(new_word)

#     print(len(new_words))
    
    return new_words
    

def generate_by_insertions(word):
    new_words = []

    for i in range(len(word)):
        for l in LOWER_CASE_LETTERS:
            new_word = word[:i] + l + word[i:]
            new_words.append(new_word)
            
#     print(len(new_words))
    
    return new_words

In [16]:
from itertools import chain

In [17]:
def generate_words(test_word):
    gen_by_substitutions = generate_by_substitutions(test_word)
    gen_by_deletions = generate_by_deletions(test_word)
    gen_by_insertions = generate_by_insertions(test_word)
    
    generated_words = gen_by_substitutions + gen_by_deletions + gen_by_insertions

    return generated_words

In [18]:
def generative_spell_checker(test_word):
    print("checking: " + test_word)
    if len(test_word) < 2:
        print("  result: " + test_word)
        return test_word
    
    generated_words = generate_words(test_word)
    generated_words += [test_word]
    generated_words_cleaned = [word for word in generated_words if word in DICTIONARY]

    if not len(generated_words_cleaned):
        generated_words = list(chain.from_iterable([generate_words(word) for word in generated_words]))
        generated_words += [test_word]
        generated_words_cleaned = [word for word in generated_words if word in DICTIONARY]
        
    print("    debug| words: %s" % (generated_words_cleaned))
    
    best_match = most_frequent(generated_words_cleaned)
    print("  result: " + best_match)
    print()
    return best_match
    

#### Тестируем

In [19]:
for test_word in X_test:
    generative_spell_checker(test_word)

checking: оцинил
    debug| words: ['оценил']
  result: оценил

checking: роботу
    debug| words: ['работу', 'робота', 'роботы', 'роботе', 'робот', 'роботу']
  result: работу

checking: новвых
    debug| words: ['новых', 'новых']
  result: новых

checking: самалетав
    debug| words: ['сабалета', 'самолетов', 'самолетам', 'самолетах', 'самолета', 'самолетов', 'самолетам', 'самолетах', 'сабалета', 'самолета']
  result: самолета

checking: и
  result: и
checking: виртолтов
    debug| words: ['вертолётов', 'вертолетов', 'вертолётов', 'вертолетов']
  result: вертолетов

checking: в
  result: в
checking: сирийи
    debug| words: ['сирицки', 'сирицки', 'сирии', 'сирии', 'сирии', 'сирии', 'сирии', 'сирии']
  result: сирии

